In [1]:
exp_num_dists = 10
imbalance = 150*(10**10)
eff_gap = -0.105
n = 10
D = 20

In [2]:
from IPython.display import display
import math
import pandas as pd
import random

rand = lambda: random.randint(100, 255)
colors = ['background-color: #%02X%02X%02X' % (rand(), rand(), rand()) for i in range(D)]

def color(val):
    return colors[int(val)]
def visualize_districts(dists):
    dist_graph = []
    for k, v in dists.items():
        dist_graph.append(v)
    dist_matrix = []
    for i in range(n):
        dist_matrix.append([])
        for j in range(n):
            dist_matrix[i].append('@')
    for index, district in enumerate(dist_graph):
        for block in district:
            x, y = get_coordinates(block, n)
            dist_matrix[x][y] = index
    df = pd.DataFrame(dist_matrix)
    # apply the colors to style for columns A and B
    style = df.style.apply(lambda x: [color(s) for s in x])
    display(style)
def get_neighbors(dist_coord):
    x, y = dist_coord
#     return (x-1)*n + y, (x+1)*n + y, (x*n)+y+1, x*n + (y-1)
    return (x-1, y), (x+1, y), (x, y+1), (x,y-1)
def get_coordinates(block, n):
    return (math.floor(block/n), block%n)

In [3]:
# setup/import data
import json
from pprint import pprint
with open('p6/voters.json') as data_file:
    data_item = json.load(data_file)
    
voters_a = data_item['voters_by_block']['party_A']
voters_b = data_item['voters_by_block']['party_B']
blocks_pop = [a + b for a, b in zip(voters_a, voters_b)]

blocks_data = {}
blocks_assigned = {}
for i in range(n):
    blocks_data[i] = {}
    blocks_assigned[i] = {}
for i in range(100):
    block = voters_a[i], voters_b[i]
    x, y = get_coordinates(i, n)
    blocks_data[x][y] = block
    blocks_assigned[x][y] = False



In [154]:
from scipy import stats
# dists will be an array of labels i.e 10, or the block at (1, 0)
def get_block_vote_split(block):
    x, y = get_coordinates(block, n)
    a, b = blocks_data[x][y]
    return a, b
def get_block_pop(block):
    a, b = get_block_vote_split(block)
    return a + b
def get_dist_vote_split(dist):
    blocks_split = [get_block_vote_split(block) for block in dist]
    a = sum([block[0] for block in blocks_split])
    b = sum([block[1] for block in blocks_split])
    return a, b
def dist_population(dist): 
    return sum([get_block_pop(block) for block in dist])
def mean_dist_pop(dists):
    mean = sum([dist_population(dist) for dist_num, dist in dists.items()])/len(dists)
    return mean
def dist_pop_imbalance(dists):
    mean = mean_dist_pop(dists)
    imbalance = sum([(dist_population(dist) - mean)**2 for dist_num, dist in dists.items()])
    return imbalance
def expected_eff_gap(dists, total_pop):
    #First, compute the number of wasted votes for both parties.
    #Then take the subtract the number of wasted votes for party B 
    #from the number of wasted votes for party A and divide that number by total population. 
    #A largely negative value is associated with gerrymandering in favor of party A.
    wasted_dists = [calc_wasted_votes(dist) for dist_num, dist in dists.items()]
    wasted_A = sum([a for (a, b) in wasted_dists])
    wasted_B = sum([b for (a, b) in wasted_dists])
    return (wasted_A - wasted_B)/total_pop
def calc_wasted_votes(dist):
    # a won
    a, b = get_dist_vote_split(dist)
    if (win_probability(a, b) > 0.50):
        return a - (a+b)/2, b
    # b won
    else:
        return a, b - (a+b)/2
def exp_votes_for_a(pop_A, pop_B):
    # E[X] = n*p
    # E[X] + E[Y]/X + Y
    return (.60*pop_A + .40*pop_B)/(pop_A + pop_B)

def win_probability(na, nb):
    mean = .6 * na + .4 * nb
    variance = (na + nb) * .4 * .6
    z = ((na + nb) / 2.0 - mean) / (variance ** 0.5)
    return 1 - stats.norm.cdf(z)

# N(μ,σ^2)

def exp_num_districts(dists):
    dists_splits = [get_dist_vote_split(dist) for dist_num, dist in dists.items()]
    probs = [win_probability(a, b) for (a, b) in dists_splits]
#     print(probs)
    return sum(probs)

In [144]:
total_pop = sum(voters_a + voters_b)
print(voters_b[0]+voters_a[0])


19290


In [6]:
districts_left = 20

In [7]:
import random
from itertools import combinations
from skimage import morphology
import numpy as np

def connected(n, dist):
    grid = np.zeros((n,n))
#     blocks = [for block in dist]
    for block in dist:
        grid[get_coordinates(block, n)] = 1
    numlabels = np.max(morphology.label(grid,4))
    if numlabels == 1:
        return True
    else:
        return False

# check if adding block to dist will result in a valid dist
def is_valid_dist(dist, block):
    if len(dist) == 0: 
        return True
#     block_coord = get_coordinates(block, n)
#     neighbors = get_neighbors(block_coord)
# #     print(neighbors)
#     # if the block has a neighbor in dist, then it's reachable
#     coords = [get_coordinates(b, n) for b in dist]
# #     for c in coords:
# #         for neigh in neighbors:
# #             if neigh[0] == c[0] and neigh[1] == c[1]:
# #                 return True
# #     return False
#     block_neighbor_valid = [neighbor in coords for neighbor in neighbors]
#     return (True in block_neighbor_valid)
    temp_dist = copy.deepcopy(dist)
    temp_dist.append(block)
    return connected(n, temp_dist)

def is_valid_configuration(config):
#     print('CHECKING VALID CONFIG')
#     visualize_districts(dists)
#     print("-" * 40)
    temp_dists = copy.deepcopy(config)
    total_blocks = 0
    valid = True
    for i in range(D):
        if len(temp_dists[str(i)]) == 0:
            print('got here')
            return False
        dist = temp_dists[str(i)]
        for block in dist:
            temp_dist = copy.deepcopy(dist)
            temp_dist.remove(block)
            valid = is_valid_dist(temp_dist, block)
            if valid == False:
                return False
        total_blocks += len(dist)
    return valid and (total_blocks == n**2)

# def best_neighbors(dists):
#     possibleSwaps = list(combinations(list(dists), 2))
#     allNeighbors = []
#     for swap in possibleSwaps:
#         allNeighbors.append(tour.move_city(tour.index(swap[0]),tour.index(swap[1])))
    # TODO: implement if needed
#     return allNeighbors

def assign_blocks_to_district(blocks):
    districts = {}
    blocks_assigned = {}
    has_empty_dist = True in [len(v)== 0 for k,v in districts.items()]
    for i in range(D):
        districts[str(i)] = []
    for i in range(n):
        blocks_assigned[i] = {}
        for j in range(n):
            blocks_assigned[i][j] = False
    for i in range(n*n):
        x, y = get_coordinates(i, n)
        while not blocks_assigned[x][y]:
            a, b = get_block_vote_split(i)
            # randomly assign as long as its valid
            if has_empty_dist:
                for k,v in districts.items():
                    if len(v) == 0:
                        blocks_assigned[x][y] = True
                        districts[str(k)].append(i)
            else:
                randDistNum = random.randint(0, D-1)
                dist = districts[str(randDistNum)]
                if is_valid_dist(dist, i):
                    blocks_assigned[x][y] = True
                    dist.append(i)
    return districts

In [8]:
# visualize_districts(dists)
import copy
def change_random_block(temp_dists):
    # Not working properly???
#     original_dists = copy.deepcopy(dists)
#     visualize_districts(dists)
#     print("-" * 40)
    tried_dists = []
    randDistStart = random.randint(0, len(temp_dists)-1)
    while len(temp_dists[str(randDistStart)]) == 1:
        randDistStart = random.randint(0, len(temp_dists)-1)
    randBlock = random.choice(temp_dists[str(randDistStart)])
    randDistTo = random.randint(0, len(temp_dists)-1)
    while randDistStart == randDistTo:
        randDistTo = random.randint(0, len(temp_dists)-1)
    coords = get_coordinates(randBlock, n)
    valid = False
    while valid == False:
        if randDistTo in tried_dists == False:
            tried_dists.append(randDistTo)
        if len(tried_dists) == len(temp_dists):
            randBlock = random.choice(temp_dists[str(randDistStart)])
            tried_dists = []
        randDistTo = random.randint(0, len(temp_dists)-1)
        if is_valid_dist(temp_dists[str(randDistTo)], randBlock):
            temp_dists[str(randDistStart)].remove(randBlock)
            temp_dists[str(randDistTo)].append(randBlock)
            if is_valid_configuration(temp_dists) == False:
                temp_dists[str(randDistStart)].append(randBlock)
                temp_dists[str(randDistTo)].remove(randBlock)
            else:
                valid = True
    return temp_dists

def cost(dists):
    return ((exp_num_dists-exp_num_districts(dists))*10)**2 + ((dist_pop_imbalance(dists)-imbalance)/1000)**2 + ((expected_eff_gap(dists, total_pop))*100000000)

def get_random_block(temp_dists):
    randDistStart = random.randint(0, len(temp_dists)-1)
    while len(temp_dists[str(randDistStart)]) == 1:
        randDistStart = random.randint(0, len(temp_dists)-1)
    return random.choice(temp_dists[str(randDistStart)]), randDistStart

def get_best_neighbor(randBlock, randStart, temp_dists):
    randDistStart = random.randint(0, len(temp_dists)-1)
#     while len(temp_dists[str(randDistStart)]) == 1:
#         randDistStart = random.randint(0, len(temp_dists)-1)
    possible_districts = list(range(D))
    possible_districts.remove(randDistStart)
    valid_dists = {}
    while len(valid_dists) < 1:
#         randBlock = random.choice(temp_dists[str(randDistStart)])
        for dist in possible_districts:
            temp = copy.deepcopy(temp_dists)
            if is_valid_dist(temp[str(dist)], randBlock):
                temp[str(dist)].append(randBlock)
                temp[str(randStart)].remove(randBlock)
                if is_valid_configuration(temp):
                    valid_dists[str(dist)] = cost(temp)
        if len(valid_dists) < 1:
            randBlock = random.choice(temp_dists[str(randDistStart)])
    configs = valid_dists.items()
    min_dist = min(configs, key= lambda t: t[1])
    temp_dists[str(min_dist[0])].append(randBlock)
    temp_dists[str(randStart)].remove(randBlock)
    return temp_dists, randBlock

def get_best_block(temp_dists):
    randDistStart = random.randint(0, len(temp_dists)-1)
    while len(temp_dists[str(randDistStart)]) == 1:
        randDistStart = random.randint(0, len(temp_dists)-1)
    possible_districts = list(range(D))
    possible_districts.remove(randDistStart)
    valid_dists = {}
    while len(valid_dists) < 1:
        randBlock = random.choice(temp_dists[str(randDistStart)])
        for dist in possible_districts:
            temp = copy.deepcopy(temp_dists)
            if is_valid_dist(temp[str(dist)], randBlock):
                temp[str(dist)].append(randBlock)
                temp[str(randDistStart)].remove(randBlock)
                if is_valid_configuration(temp):
                    valid_dists[str(dist)] = cost(temp)
    configs = valid_dists.items()
    min_dist = min(configs, key= lambda t: t[1])
    temp_dists[str(min_dist[0])].append(randBlock)
    temp_dists[str(randDistStart)].remove(randBlock)
    return temp_dists, randBlock

# while exp_num_districts(dists) < 9.9 or dist_pop_imbalance(dists) > imbalance or expected_eff_gap(dists, total_pop) < eff_gap:
#     orig_dists = copy.deepcopy(dists)
#     new_dists = change_random_block(dists)
# #     new_imbalance = dist_pop_imbalance(new_dists)
#     new_cost = cost(new_dists)
#     old_cost = cost(orig_dists)
#     if is_valid_configuration(new_dists) == False:
#         print("invalid configuration!!!")
#     if new_cost <= old_cost:
#         current_best_imbalance = new_imbalance
#         print('cost:', new_cost)
#         print('num districts progress: ', exp_num_districts(dists)/exp_num_dists * 100)
#         print('imbalance: ', abs(dist_pop_imbalance(dists)-imbalance))
#         print('eff gap: ', expected_eff_gap(dists, total_pop))
#         print("-" * 40)
#         dists = new_dists
#     else:
#         dists = orig_dists
# print('imbalance', current_best_imbalance)
# print(is_valid_configuration(new_dists))
# visualize_districts(dists)

In [175]:
def cost(dists):
    return ((exp_num_dists-exp_num_districts(dists)))*10000000
tried_blocks= {}
dists = assign_blocks_to_district(blocks_data)
current_best_imbalance = dist_pop_imbalance(dists)
current_best_exp = exp_num_districts(dists)
while current_best_exp < 10:
    orig_dists = copy.deepcopy(dists)
    randBlock, randStart = get_random_block(dists)
    while randBlock in tried_blocks:
        randBlock, randStart = get_random_block(dists)
    new_dists, tried_block = get_best_neighbor(randBlock, randStart, dists)
    tried_blocks[str(tried_block)] = True
    new_exp = exp_num_districts(new_dists)
    if new_exp > current_best_exp:
        current_best_exp = new_exp
        print(current_best_exp)
        dists = new_dists
    else:
        dists = orig_dists
    if (len(tried_blocks) > 90):
        print('prob at a local max')
        break
df = visualize_districts(dists)
df

7.060343402192591
7.457599027470901
8.44667476658331
8.446921928308152
9.26457193175949
9.396910364265864
10.35288639198287


,0,1,2,3,4,5,6,7,8,9
0,0,2,3,6,6,7,17,13,12,9
1,2,2,2,14,18,10,1,15,16,8
2,2,11,11,11,18,10,4,15,16,5
3,2,19,11,11,18,18,18,18,16,5
4,2,19,11,11,18,18,18,18,18,18
5,2,2,11,11,18,18,18,18,18,18
6,2,2,2,11,18,18,18,18,18,18
7,2,2,2,2,2,18,18,18,18,18
8,2,2,2,2,2,18,18,18,18,18
9,2,2,2,2,2,2,18,18,18,18


In [245]:
visualize_districts(dists)
print(exp_num_districts(dists))
print(dist_pop_imbalance(dists))
print(expected_eff_gap(dists, total_pop))
print(exp_num_districts(dists) >= exp_num_dists, dist_pop_imbalance(dists) <= imbalance, expected_eff_gap(dists, total_pop) >= eff_gap)

,0,1,2,3,4,5,6,7,8,9
0,0,0,3,17,17,17,17,8,8,8
1,0,0,17,17,7,7,13,9,15,8
2,0,14,6,6,7,12,4,2,16,2
3,0,14,6,7,7,1,4,2,2,2
4,0,14,6,7,7,10,4,18,2,5
5,0,0,6,7,7,11,11,18,2,2
6,0,0,6,6,7,11,18,18,2,2
7,0,6,6,6,7,11,2,2,2,2
8,0,6,6,7,7,11,2,2,2,2
9,19,6,6,6,11,11,11,11,11,2


12.18771180953429
8009349320370.95
-0.07834036238554325
True False True


In [292]:
saved_dist = copy.deepcopy(dists)
visualize_districts(saved_dist)
print(exp_num_districts(saved_dist))
print(dist_pop_imbalance(saved_dist))
print(expected_eff_gap(saved_dist, total_pop))
print(exp_num_districts(saved_dist) >= exp_num_dists, dist_pop_imbalance(saved_dist) <= imbalance, expected_eff_gap(saved_dist, total_pop) >= eff_gap)

,0,1,2,3,4,5,6,7,8,9
0,17,17,3,3,13,13,8,8,8,8
1,17,17,3,3,13,1,9,16,16,16
2,0,17,14,3,7,1,12,16,15,5
3,0,17,14,3,7,4,4,4,4,5
4,0,17,14,3,7,10,4,10,4,5
5,0,17,14,6,7,10,10,10,2,5
6,0,0,6,6,7,18,10,10,2,5
7,0,19,6,6,7,18,2,2,2,11
8,19,19,6,18,18,18,2,11,2,11
9,19,6,6,6,18,11,11,11,11,11


10.000000462877905
1487255695564.95
-0.09133119636040318
True True True


In [289]:
import random
from math import exp, inf

temp = 50

temp_decay = 0.95
runs = 20

def random_temp(delta):
    global temp
    probability = exp(delta/temp)
    temp *= temp_decay
    return (random.random() <= probability)
def cost(dists):
    if strategy == 'dist':
        return ((exp_num_dists-exp_num_districts(dists)))*10000000
    elif strategy == 'gap':
        return (expected_eff_gap(dists, total_pop)-eff_gap)*-10000000000000
    elif strategy == 'imbalance':
        return dist_pop_imbalance(dists) - imbalance
    else:
        return ((exp_num_dists-exp_num_districts(dists)))**2*100000000 + (expected_eff_gap(dists, total_pop)-eff_gap)*-100000000000000000 + (dist_pop_imbalance(dists) - imbalance)
#     return ((exp_num_dists-exp_num_districts(dists)))*100000

dists = copy.deepcopy(saved_dist)
# dists = assign_blocks_to_district(blocks_data)
current_best_exp = exp_num_districts(dists)
current_imbalance = dist_pop_imbalance(dists)
current_gap = expected_eff_gap(dists, total_pop)

tried_blocks = {}

bestState = copy.deepcopy(dists)
strategy = 'dist'
bestCost = cost(bestState)

# for i in range(0, runs):
#     global temp
#     temp = 50
#     if i % 10 == 0:
#         print('run ', i)
#     dists = copy.deepcopy(saved_dist)
#     current_best_exp = exp_num_districts(dists)
#     current_imbalance = dist_pop_imbalance(dists)
#     current_cost = cost(dists)
while current_best_exp < 10:
    orig_dists = copy.deepcopy(dists)
    randBlock, randStart = get_random_block(dists)
    while randBlock in tried_blocks:
        randBlock, randStart = get_random_block(dists)
    new_dists, tried_block = get_best_neighbor(randBlock, randStart, dists)
    tried_blocks[str(tried_block)] = True
    new_exp_num = exp_num_districts(new_dists)
    new_imbalance = dist_pop_imbalance(new_dists)
    new_eff_gap = expected_eff_gap(new_dists, total_pop)
#     if current_cost > new_cost:
    if new_exp_num > current_best_exp and (new_imbalance < imbalance and new_eff_gap > eff_gap):
        current_best_exp = new_exp_num
        current_gap = new_eff_gap
        current_imbalance = new_imbalance
        new_cost = cost(new_dists)
        current_cost = cost(new_dists)
        print(current_best_exp)
        print(current_gap)
        print(current_imbalance)
        if new_cost < bestCost:
            print('new best!!!')
            bestCost = new_cost
            bestState = new_dists
        tried_blocks = {}
        dists = new_dists
#         visualize_districts(dists)
    else:
        if random_temp(new_exp_num-10):
            current_best_exp = new_exp_num
            current_best_exp = new_exp_num
            current_gap = new_eff_gap
            current_imbalance = new_imbalance
            tried_blocks = {}
            dists = new_dists
        else:
            dists = orig_dists
    if (len(tried_blocks) > 90):
        print('prob at a local max')
        break

In [291]:
strategy = 'imbalance'

dists = copy.deepcopy(saved_dist)
current_best_exp = exp_num_districts(dists)
current_imbalance = dist_pop_imbalance(dists)
current_gap = expected_eff_gap(dists, total_pop)
tried_blocks = {}

temp = 50

temp_decay = 0.95
runs = 20

def random_temp(delta):
    global temp
    probability = exp(delta/temp)
    temp *= temp_decay
    return (random.random() <= probability)

if strategy == 'imbalance':
    while current_imbalance > imbalance:
        orig_dists = copy.deepcopy(dists)
        randBlock, randStart = get_random_block(dists)
        while randBlock in tried_blocks:
            randBlock, randStart = get_random_block(dists)
        new_dists, tried_block = get_best_neighbor(randBlock, randStart, dists)
        tried_blocks[str(tried_block)] = True
        new_exp_num = exp_num_districts(new_dists)
        new_imbalance = dist_pop_imbalance(new_dists)
        if new_imbalance < current_imbalance and new_exp_num > 10 and expected_eff_gap(new_dists, total_pop) > eff_gap:
            current_imbalance = new_imbalance
            tried_blocks = {}
            print(dist_pop_imbalance(dists))
            dists = new_dists
            bestState = new_dists
    #         visualize_districts(dists)
        else:
#             if random_temp(new_imbalance-current_imbalance):
#                 current_best_exp = new_exp_num
#                 current_best_exp = new_exp_num
#                 current_gap = new_eff_gap
#                 current_imbalance = new_imbalance
#                 tried_blocks = {}
#                 dists = new_dists
#             else:
#                 dists = orig_dists
            dists = orig_dists
        if (len(tried_blocks) > 90):
            print('prob at a local max')
            break
if strategy == 'gap':
    while current_gap < 0:
        orig_dists = copy.deepcopy(dists)
        randBlock, randStart = get_random_block(dists)
        while randBlock in tried_blocks:
            randBlock, randStart = get_random_block(dists)
        new_dists, tried_block = get_best_neighbor(randBlock, randStart, dists)
        tried_blocks[str(tried_block)] = True
        new_exp_num = exp_num_districts(new_dists)
        new_imbalance = dist_pop_imbalance(new_dists)
        new_gap = expected_eff_gap(new_dists, total_pop)
        if new_exp_num > 10 and new_imbalance < current_imbalance and new_gap > eff_gap:
            current_gap = new_gap
            tried_blocks = {}
            print(new_gap)
            dists = new_dists
            bestState = new_dists

    #         visualize_districts(dists)
        else:
            dists = orig_dists
        if (len(tried_blocks) > 90):
            print('prob at a local max')
            break

            
            


1904261480758.95
1853391101236.95
1778935130368.95
1635822258928.95
1630877500048.95
1608016949044.95
1487255695564.95


In [1100]:
saveee = copy.deepcopy(saved_dist)
saveee

{'0': [68, 49, 56, 57, 59, 58, 78],
 '1': [85],
 '2': [34],
 '3': [75, 64, 74, 83, 73, 65, 66, 55],
 '4': [31, 50, 10, 20, 30, 0, 40, 1],
 '5': [60, 61, 51, 70],
 '6': [15, 17, 16],
 '7': [19, 29, 6, 9, 5, 7, 8],
 '8': [37, 39, 27, 38, 28, 48, 18, 47],
 '9': [89, 69, 79],
 '10': [53, 72, 62, 82, 52, 63],
 '11': [90],
 '12': [21],
 '13': [26],
 '14': [71],
 '15': [23, 2, 4, 22, 12, 11, 3],
 '16': [32],
 '17': [93, 92, 91, 81, 80],
 '18': [41, 54, 45, 42, 44, 24, 43, 13, 35, 46, 25, 36, 14, 33],
 '19': [98, 86, 97, 94, 99, 77, 87, 76, 96, 67, 84, 95, 88]}

In [293]:
def convert_to_solution(final_dists):
    solution = []
    for k, v in final_dists.items():
        solution.append(v)
    print(solution)
convert_to_solution(dists)

[[30, 50, 60, 40, 70, 61, 20], [25, 15], [58, 88, 86, 68, 76, 77, 78], [12, 13, 43, 2, 23, 33, 3], [35, 48, 46, 38, 37, 36], [39, 49, 59, 29, 69], [63, 92, 73, 53, 91, 72, 93, 62, 82], [34, 24, 64, 74, 44, 54], [7, 8, 9, 6], [16], [55, 67, 57, 45, 47, 66, 56], [95, 89, 96, 98, 97, 99, 87, 79], [26], [5, 4, 14], [32, 42, 22, 52], [28], [19, 18, 17, 27], [41, 11, 0, 51, 10, 21, 31, 1], [75, 65, 83, 84, 85, 94], [90, 81, 80, 71]]


In [ ]:
print(dists)
print(exp_num_districts(dists))
print(dist_pop_imbalance(dists))
print(expected_eff_gap(dists, total_pop))
print(exp_num_districts(dists) >= exp_num_dists, dist_pop_imbalance(dists) <= imbalance, expected_eff_gap(dists, total_pop) >= eff_gap)
print(dist_pop_imbalance(dists))
print(expected_eff_gap(dists, total_pop))
current_best_imbalance = 10000000000000000
while current_best_imbalance > imbalance:
    original_dists = dict(dists)
#     new_dists = change_random_block(dists)
    new_dists = assign_blocks_to_district(blocks)
    new_imbalance = dist_pop_imbalance(new_dists)
    if new_imbalance <= current_best_imbalance:
        current_best_imbalance = new_imbalance
        print('imbalance', current_best_imbalance)
        dists = new_dists
    else:
        dists = original_dists
print(dists)
print("DONE WITH IMBALANCE")
current_best_exp = 0
exp_num_dists = 10
while current_best_exp < exp_num_dists:
    original_dists = dict(dists)
#     new_dists = change_random_block(dists)
    new_dists = assign_blocks_to_district(blocks)
    new_exp_num = exp_num_districts(new_dists)
    if ((dist_pop_imbalance(new_dists) <= imbalance) and (current_best_exp <= new_exp_num)):
        current_best_exp = new_exp_num
        print('num districts', new_exp_num)
        dists = new_dists
    else:
        dists = original_dists
print(exp_num_districts(dists), dist_pop_imbalance(dists))
print(dists)
print("DONE WITH DISTRICTS")
visualize_districts(dists)
current_best_gap = -1
while current_best_gap < eff_gap:
    original_dists = dict(dists)
#     new_dists = change_random_block(dists)
    new_dists = assign_blocks_to_district(blocks)
    new_gap = expected_eff_gap(new_dists, total_pop)
    if ((exp_num_districts(new_dists) >= exp_num_dists) and (dist_pop_imbalance(new_dists) <= imbalance) and (current_best_gap <= new_gap)):
        current_best_gap = new_gap
        print('eff gap', current_best_gap)
        dists = new_dists
    else:
        dists = original_dists
print(dists)
print("DONE WITH GAP")
visualize_districts(dists)
print(exp_num_districts(dists) >= exp_num_dists, dist_pop_imbalance(dists) <= imbalance, expected_eff_gap(dists, total_pop) >= eff_gap)

{0: [16, 26, 27, 28, 36, 37, 38, 39, 46, 47, 48, 49, 57, 58, 59, 69], 1: [25], 2: [19, 29], 3: [20, 21, 30, 31, 40, 41, 50, 51, 60, 61, 62, 63, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 4: [7], 5: [2], 6: [22, 23, 32, 33, 34, 35, 42, 43, 44, 45, 52, 53, 54, 55, 56, 64, 65, 66, 67, 68], 7: [0, 1], 8: [9], 9: [5], 10: [17], 11: [3], 12: [10, 11, 12], 13: [13], 14: [24], 15: [4], 16: [15], 17: [6], 18: [8, 18], 19: [14]}
9.527809495935779
22929852147822.95
0.5791657816320346
False False True
22929852147822.95
0.5791657816320346
imbalance 15280674659158.95
imbalance 15181689755252.95
imbalance 13774456849792.951
imbalance 12706975212456.95
imbalance 10378049222896.95
imbalance 8969438807200.95
imbalance 8722516932160.95
imbalance 8270358793216.95
imbalance 7476829706964.949
imbalance 6768174844576.951
imbalance 6323838190466.951


In [ ]:
# key is dist, value is true if we've tried it
combination_tried = {}
# dists is an array of length D, of tuples (num_party_A, num_party_B) in district i
dists = []
# win condition


In [ ]:
print(exp_num_districts(dists) >= exp_num_dists, dist_pop_imbalance(dists) <= imbalance, expected_eff_gap(dists, total_pop) >= eff_gap)

In [1]:
print(dist_pop_imbalance(dists))
print(imbalance)

NameError: name 'dist_pop_imbalance' is not defined

In [4]:
11690978651180.949 - 150*(10**10)

10190978651180.95

In [515]:
print(exp_num_districts(dists))

10.002898155019142


In [518]:
print(expected_eff_gap(dists, total_pop))

0.26826444716245207


In [236]:
def cost(dists):
    return ((exp_num_dists-exp_num_districts(dists))*100000)**2 + ((dist_pop_imbalance(dists)-imbalance))**2 + ((expected_eff_gap(dists, total_pop))*100000000000000)

current_best_exp = 0
while current_best_exp < 9.9:
    original_dists = dict(dists)
    new_dists = change_random_block(dists)
    new_exp_num = exp_num_districts(new_dists)
    if (current_best_exp <= new_exp_num):
        current_best_exp = new_exp_num
        print('num districts', new_exp_num)
        dists = new_dists
    else:
        dists = original_dists
print(exp_num_districts(dists), dist_pop_imbalance(dists))

print("DONE WITH DISTRICTS")
current_cost = 100000000000000001000000000000000010000000000000000
while exp_num_districts(dists) < 9.9 or dist_pop_imbalance(dists) > imbalance or expected_eff_gap(dists, total_pop) < eff_gap:
    original_dists = dict(dists)
    new_dists = change_random_block(dists)
    new_cost = cost(new_dists)
    if exp_num_districts(new_dists) >= 9.9 and new_cost <= current_cost:
        current_cost = new_cost
        print('cost', new_cost)
        print('num districts', exp_num_districts(new_dists))
        print('imbalance', dist_pop_imbalance(new_dists))
        print('eff gap', expected_eff_gap(new_dists, total_pop))
        dists = new_dists
    else:
        dists = original_dists
print("DONZOOOO")
print(dists)

num districts 9.60839342408302
num districts 9.620063651854714
num districts 9.620063651854714
num districts 9.620063651854714
num districts 9.642175336434914
num districts 9.642175336434914
num districts 9.642175336434914


KeyboardInterrupt: 

In [126]:
dist_graph = []
for k, v in dists.items():
    dist_graph.append(v)
dist_matrix = []
for i in range(n):
    dist_matrix.append([])
    for j in range(n):
        dist_matrix[i].append('@')
for index, district in enumerate(dist_graph):
    for block in district:
        x, y = get_coordinates(block, n)
        dist_matrix[x][y] = index
p = pd.DataFrame(dist_matrix)
p
# def highlight_cells(x):
#     df = x.copy()
#     #set default color
#     #df.loc[:,:] = 'background-color: papayawhip' 
#     df.loc[:,:] = '' 
#     #set particular cell colors
#     df.iloc[0,0] = 'background-color: red'
#     return df 

# t = data.style.apply(highlight_cells, axis=None)


,0,1,2,3,4,5,6,7,8,9
0,3,2,4,6,5,9,10,17,1,12
1,14,2,0,16,18,13,8,7,19,12
2,11,2,0,16,15,13,8,8,19,12
3,11,2,0,15,15,15,8,15,19,12
4,11,11,0,15,8,8,8,15,15,12
5,11,0,0,15,8,8,8,15,15,15
6,11,11,11,15,15,15,15,15,15,15
7,11,11,11,15,15,15,15,15,15,15
8,11,11,11,11,15,15,15,15,15,15
9,11,11,11,11,11,15,15,15,15,15
